In [1]:
import re
import pandas as pd
import tensorflow as tf
from src.models import *
from src.backbone import *

In [2]:
train_labels = pd.read_csv('./data/IIIT5K/train.csv')
train_labels.head()

,Unnamed: 0,image,text
0,0,./data/IIIT5K/train/1009_2.png,YOU
1,1,./data/IIIT5K/train/1017_1.png,RESCUE
2,2,./data/IIIT5K/train/1017_2.png,MISSION
3,3,./data/IIIT5K/train/1021_1.png,HOME
4,4,./data/IIIT5K/train/1023_1.png,BORDER


In [3]:
texts = ['¿' + i + '?' for i in train_labels['text']]
texts = [' '.join(i) for i in texts]
texts[:5]

['¿ Y O U ?',
 '¿ R E S C U E ?',
 '¿ M I S S I O N ?',
 '¿ H O M E ?',
 '¿ B O R D E R ?']

In [4]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
tokenizer.fit_on_texts(texts)

In [5]:
images = list(train_labels['image'])
text_labels = tokenizer.texts_to_sequences(texts)
text_labels = tf.keras.preprocessing.sequence.pad_sequences(text_labels, padding='post')

In [6]:
text_labels[:5]

array([[ 1, 22,  5, 16,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0],
       [ 1,  8,  3, 10, 12, 16,  3,  2,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0],
       [ 1, 15,  7, 10, 10,  7,  5,  9,  2,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0],
       [ 1, 13,  5, 15,  3,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0],
       [ 1, 20,  5,  8, 14,  3,  8,  2,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0]], dtype=int32)

In [7]:
@tf.function
def map_function(image_file, text):
    image = tf.io.read_file(image_file)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, (96, 96))
    image = tf.cast(image, tf.float32)
    image = tf.keras.applications.resnet.preprocess_input(image)
    return image, text

In [8]:
dataset = tf.data.Dataset.from_tensor_slices((images, text_labels))
dataset = dataset.shuffle(128)
dataset = dataset.map(map_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset = dataset.batch(8)
dataset

<BatchDataset shapes: ((None, 96, 96, 3), (None, 24)), types: (tf.float32, tf.int32)>